In [ ]:
!pip install -q gradio requests gTTS


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.2 MB/s eta 0:00:00


In [ ]:
pip install gradio requests pyttsx3


In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip install gradio requests gTTS


In [ ]:
!pip install gradio requests gTTS


In [ ]:
import gradio as gr
import requests
from gtts import gTTS
import tempfile

API_KEY = "3adb597335b3456b896666fa9a244786"  # Replace with your actual key if needed

COUNTRIES = {
    "All": None,
    "United Arab Emirates": "ae", "Argentina": "ar", "Austria": "at", "Australia": "au", "Belgium": "be",
    "Bulgaria": "bg", "Brazil": "br", "Canada": "ca", "Switzerland": "ch", "China": "cn", "Colombia": "co",
    "Cuba": "cu", "Czech Republic": "cz", "Germany": "de", "Egypt": "eg", "France": "fr", "United Kingdom": "gb",
    "Greece": "gr", "Hong Kong": "hk", "Hungary": "hu", "Indonesia": "id", "Ireland": "ie", "Israel": "il",
    "India": "in", "Italy": "it", "Japan": "jp", "South Korea": "kr", "Lithuania": "lt", "Latvia": "lv",
    "Morocco": "ma", "Mexico": "mx", "Malaysia": "my", "Nigeria": "ng", "Netherlands": "nl", "Norway": "no",
    "New Zealand": "nz", "Philippines": "ph", "Poland": "pl", "Portugal": "pt", "Romania": "ro", "Serbia": "rs",
    "Russia": "ru", "Saudi Arabia": "sa", "Sweden": "se", "Singapore": "sg", "Slovenia": "si", "Slovakia": "sk",
    "Thailand": "th", "Turkey": "tr", "Taiwan": "tw", "Ukraine": "ua", "United States": "us", "Venezuela": "ve",
    "South Africa": "za"
}

CATEGORIES = [
    "general", "business", "entertainment", "health",
    "science", "sports", "technology"
]

LANGUAGES = {
    "English": "en",
    "French": "fr",
    "German": "de",
    "Spanish": "es",
    "Arabic": "ar"
}

def text_to_speech(text):
    tts = gTTS(text=text, lang='en')
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
    tts.save(temp_file.name)
    return temp_file.name

def fetch_news(mode, category, country, count, keyword, lang_name, speak):
    country_code = COUNTRIES[country]
    lang_code = LANGUAGES.get(lang_name, "en")
    url = "https://newsapi.org/v2/top-headlines" if mode == "Top Headlines" else "https://newsapi.org/v2/everything"
    params = {
        "apiKey": API_KEY,
        "pageSize": count
    }

    if mode == "Top Headlines":
        params["category"] = category
        if country_code:
            params["country"] = country_code
    else:
        if not keyword:
            return "❗ Please enter a keyword for search.", None, None
        params["q"] = keyword
        params["language"] = lang_code

    response = requests.get(url, params=params)
    data = response.json()
    articles = data.get("articles", [])

    if not articles and mode == "Top Headlines" and country_code:
        # Retry without country filter
        params.pop("country", None)
        response = requests.get(url, params=params)
        articles = response.json().get("articles", [])

    if not articles:
        # Fallback: try everything endpoint with keyword as country name
        fallback_params = {
            "apiKey": API_KEY,
            "q": country,
            "pageSize": count,
            "language": lang_code
        }
        response = requests.get("https://newsapi.org/v2/everything", params=fallback_params)
        articles = response.json().get("articles", [])

    if not articles:
        return "❌ No news found.", None, None

    # Format news output
    speech_text = ""
    formatted_news = ""
    for i, article in enumerate(articles, 1):
        title = article.get("title", "No Title")
        url = article.get("url", "")
        desc = article.get("description", "")
        image = article.get("urlToImage", "")
        title_line = f"<a href='{url}' target='_blank'>{title}</a>" if url else title
        image_line = f"<img src='{image}' width='300'><br>" if image else ""
        desc_line = f"<p>{desc}</p>" if desc else ""
        formatted_news += f"{image_line}<b>{i}. {title_line}</b><br>{desc_line}<hr>"
        speech_text += f"{title}. "

    voice_file = text_to_speech(speech_text) if speak else None

    # Save plain text
    plain_text = "\n\n".join([f"{i+1}. {a.get('title','')}\n{a.get('description','')}" for i, a in enumerate(articles)])
    with tempfile.NamedTemporaryFile(delete=False, suffix=".txt", mode="w") as f:
        f.write(plain_text)
        text_file_path = f.name

    return formatted_news, voice_file, text_file_path

with gr.Blocks() as app:
    gr.Markdown("## 📰 News App with Country, Language, Voice, Images, and Download")

    with gr.Row():
        mode = gr.Radio(["Top Headlines", "Search by Keyword"], label="Mode", value="Top Headlines")
        lang = gr.Dropdown(choices=list(LANGUAGES.keys()), label="Language (for Search)", value="English")

    with gr.Row():
        category = gr.Dropdown(choices=CATEGORIES, label="Category", value="general")
        country = gr.Dropdown(choices=list(COUNTRIES.keys()), label="Country", value="India")
        count = gr.Slider(minimum=1, maximum=10, step=1, value=5, label="Number of Articles")

    keyword = gr.Textbox(label="Keyword (for Search mode)", placeholder="e.g. Elections, AI, Sports...")
    speak = gr.Checkbox(label="🔊 Read the news aloud", value=True)

    btn = gr.Button("Fetch News")

    html_news = gr.HTML()
    audio_output = gr.Audio(label="Voice Output", type="filepath")
    file_output = gr.File(label="📄 Download News as Text")

    btn.click(
        fn=fetch_news,
        inputs=[mode, category, country, count, keyword, lang, speak],
        outputs=[html_news, audio_output, file_output]
    )

app.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://446d4777b200431d86.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install gradio requests

import gradio as gr
import requests

# 🔑 Your NewsAPI Key
NEWS_API_KEY = "3adb597335b3456b896666fa9a244786"

# Store context (e.g., last searched topic)
chat_context = {
    "last_topic": None,
    "page": 1
}

def fetch_news(topic, page=1):
    url = (
        f"https://newsapi.org/v2/everything?"
        f"q={topic}&apiKey={NEWS_API_KEY}&language=en&pageSize=5&page={page}"
    )
    res = requests.get(url)
    if res.status_code != 200:
        return None, "⚠️ API error. Try again later."

    data = res.json()
    articles = data.get("articles", [])
    if not articles:
        return None, "❌ No news found."

    news_text = ""
    for i, article in enumerate(articles, 1):
        title = article.get("title", "No title")
        url = article.get("url", "#")
        news_text += f"{i}. [{title}]({url})\n"

    return articles, news_text


# Chatbot logic
def chatbot_response(message, history):
    global chat_context

    msg = message.strip().lower()

    # If user asks for updates without specifying topic
    if msg in ["more", "next", "next updates", "any updates?", "latest?", "show me more"]:
        if not chat_context["last_topic"]:
            return "❗Please enter a topic first."
        chat_context["page"] += 1
        _, response = fetch_news(chat_context["last_topic"], chat_context["page"])
        return response

    # New topic
    chat_context["last_topic"] = msg
    chat_context["page"] = 1
    _, response = fetch_news(msg, page=1)
    return response

# Gradio UI
chatbot = gr.ChatInterface(chatbot_response, title="🧠 News Chatbot", theme="soft")

chatbot.launch()


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b0f798d3d3f20b1d4e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
